<a href="https://colab.research.google.com/github/grantthiltgen/cube-density/blob/master/bloom_independent_full_analysis_large_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install shap


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 564 kB 9.6 MB/s 


In [ ]:
import collections
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, GroupShuffleSplit
from sklearn.inspection import permutation_importance
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt
import shap
plt.rcParams.update({'font.size': 22})

In [ ]:
#data_df = pd.read_csv('/content/drive/MyDrive/Bloom Data/data/pe_with_conversion_feature_engineering.zip')
data_df = pd.read_csv('/content/drive/MyDrive/Bloom Data/data/pe_with_conversion.zip')
data_df.head()

,createdAt,event,IP,Uid,CollectionID,ProductID,ImageID,Referer,URL,Country,Region,City,MetroArea,Latitude,Longitude,BrowserName,BrowserMajorVersion,BrowserVersion,OSName,OSVersion,DeviceVendor,DeviceModel,DeviceType,Background=Lifestyle,Background=Studio,BodyVisible=Both,BodyVisible=Bottom,BodyVisible=Top,Model=0,Model=1,Model=10,Model=101,Model=102,Model=103,Model=108,Model=109,Model=110,Model=112,Model=120,Model=121,Model=125,Model=135,Model=14,Model=15,Model=160,Model=19,Model=2,Model=200,Model=201,Model=202,Model=203,Model=21,Model=23,Model=24,Model=25,Model=26,Model=27,Model=28,Model=29,Model=3,Model=31,Model=32,Model=33,Model=37,Model=38,Model=4,Model=41,Model=43,Model=44,Model=46,Model=48,Model=5,Model=51,Model=53,Model=58,Model=59,Model=6,Model=60,Model=66,Model=67,Model=68,Model=72,Model=74,Model=77,Model=78,Model=8,Model=80,Model=81,Model=83,Model=84,Model=85,Model=86,Model=9,Model=N/A,Model=unknown,MoreThanOneModel=FALSE,MoreThanOneModel=TRUE,Pose=Back,Pose=Front,Pose=Side,Pose=Tilt (45),Position=N/A,Position=Sitting,Position=Standing,ProductTag=Both,ProductTag=Bottom,ProductTag=N/A,ProductTag=Top,Recognizable=FALSE,Recognizable=TRUE,ShootingStyle=Detail,ShootingStyle=Flatlay,ShootingStyle=On-figure,UniqueCameraAngle=FALSE,UniqueCameraAngle=TRUE,UniquePose=FALSE,UniquePose=TRUE,conversion,browser_,browser_Facebook,browser_Instagram,browser_Edge,browser_Opera,browser_WebKit,browser_Mobile Safari,browser_GSA,browser_Safari,browser_Firefox,browser_Samsung Browser,browser_Opera Touch,browser_Android Browser,browser_Chrome WebView,browser_Line,browser_Chrome,browser_MIUI Browser,browser_WeChat,device_vendor_,device_vendor_Huawei,device_vendor_Xiaomi,device_vendor_Fairphone,device_vendor_ASUS,device_vendor_HTC,device_vendor_Lenovo,device_vendor_Vivo,device_vendor_Apple,device_vendor_Realme,device_vendor_OnePlus,device_vendor_LG,device_vendor_Samsung,device_vendor_Google,device_vendor_Motorola,device_vendor_Sony,device_vendor_OPPO,device_vendor_Nokia,is_ref_from_collections_page,is_ref_from_another_product_page,group,rate,Background=Lifestyle_convert,Background=Studio_convert,BodyVisible=Both_convert,BodyVisible=Bottom_convert,BodyVisible=Top_convert,Model=0_convert,Model=1_convert,Model=10_convert,Model=101_convert,Model=102_convert,Model=103_convert,Model=108_convert,Model=109_convert,Model=110_convert,Model=112_convert,Model=120_convert,Model=121_convert,Model=125_convert,Model=135_convert,Model=14_convert,Model=15_convert,Model=160_convert,Model=19_convert,Model=2_convert,Model=200_convert,Model=201_convert,Model=202_convert,Model=203_convert,Model=21_convert,Model=23_convert,Model=24_convert,Model=25_convert,Model=26_convert,Model=27_convert,Model=28_convert,Model=29_convert,Model=3_convert,Model=31_convert,Model=32_convert,Model=33_convert,Model=37_convert,Model=38_convert,Model=4_convert,Model=41_convert,Model=43_convert,Model=44_convert,Model=46_convert,Model=48_convert,Model=5_convert,Model=51_convert,Model=53_convert,Model=58_convert,Model=59_convert,Model=6_convert,Model=60_convert,Model=66_convert,Model=67_convert,Model=68_convert,Model=72_convert,Model=74_convert,Model=77_convert,Model=78_convert,Model=8_convert,Model=80_convert,Model=81_convert,Model=83_convert,Model=84_convert,Model=85_convert,Model=86_convert,Model=9_convert,Model=N/A_convert,Model=unknown_convert,MoreThanOneModel=FALSE_convert,MoreThanOneModel=TRUE_convert,Pose=Back_convert,Pose=Front_convert,Pose=Side_convert,Pose=Tilt (45)_convert,Position=N/A_convert,Position=Sitting_convert,Position=Standing_convert,ProductTag=Both_convert,ProductTag=Bottom_convert,ProductTag=N/A_convert,ProductTag=Top_convert,Recognizable=FALSE_convert,Recognizable=TRUE_convert,ShootingStyle=Detail_convert,ShootingStyle=Flatlay_convert,Background=Lifestyle_nonconvert,Background=Studio_nonconvert,BodyVisible=Both_nonconvert,BodyVisible=Bottom_nonconvert,BodyVisible=Top_nonconvert,Model=0_nonconvert,Model=1_nonconvert,Model=10_nonconvert,Model=101_nonconvert,Model=102_n

In [ ]:
data_df['conversion'].value_counts()

0    758206
1     25390
Name: conversion, dtype: int64

# Add groups for training and separation

In [ ]:
Uids = data_df.Uid.unique()
UidDict = dict(zip(Uids, range(len(Uids))))

In [ ]:
data_df['Uid'] = data_df['Uid'].apply(lambda x: UidDict[x])

In [ ]:
data_df['group'] = data_df['ProductID'].astype(str) + '_' + data_df['Uid'].astype(str)

In [ ]:
groups = data_df.group.unique()
GroupDict = dict(zip(groups, range(len(groups))))
data_df['group'] = data_df['group'].apply(lambda x: GroupDict[x])

In [ ]:
data_df.head()

,Unnamed: 0,Unnamed: 0.1,createdAt,event,IP,Uid,CollectionID,ProductID,ImageID,Referer,URL,Country,Region,City,MetroArea,Latitude,Longitude,BrowserName,BrowserMajorVersion,BrowserVersion,OSName,OSVersion,DeviceVendor,DeviceModel,DeviceType,Background=Lifestyle,Background=Studio,BodyVisible=Both,BodyVisible=Bottom,BodyVisible=Top,Model=0,Model=1,Model=10,Model=101,Model=102,Model=103,Model=108,Model=109,Model=110,Model=112,Model=120,Model=121,Model=125,Model=135,Model=14,Model=15,Model=160,Model=19,Model=2,Model=200,Model=21,Model=23,Model=24,Model=25,Model=26,Model=27,Model=28,Model=29,Model=3,Model=31,Model=32,Model=33,Model=37,Model=38,Model=4,Model=41,Model=43,Model=44,Model=46,Model=48,Model=5,Model=51,Model=53,Model=58,Model=59,Model=6,Model=60,Model=66,Model=67,Model=68,Model=72,Model=74,Model=77,Model=78,Model=8,Model=80,Model=81,Model=83,Model=84,Model=85,Model=86,Model=9,Model=unknown,MoreThanOneModel=FALSE,MoreThanOneModel=TRUE,Pose=Back,Pose=Front,Pose=Side,Pose=Tilt (45),Position=Sitting,Position=Standing,ProductTag=Both,ProductTag=Bottom,ProductTag=N/A,ProductTag=Top,ProductTag_BodyVisible=Both_Both,ProductTag_BodyVisible=Both_Bottom,ProductTag_BodyVisible=Both_Top,ProductTag_BodyVisible=Bottom_Both,ProductTag_BodyVisible=Bottom_Bottom,ProductTag_BodyVisible=Bottom_Top,ProductTag_BodyVisible=N/A,ProductTag_BodyVisible=Top_Both,ProductTag_BodyVisible=Top_Bottom,ProductTag_BodyVisible=Top_Top,Recognizable=FALSE,Recognizable=TRUE,ShootingStyle=Flatlay,ShootingStyle=On-figure,UniqueCameraAngle=FALSE,UniqueCameraAngle=TRUE,UniquePose=FALSE,UniquePose=TRUE,conversion,browser_Chrome,browser_Mobile Safari,browser_Instagram,browser_Edge,browser_Facebook,browser_GSA,browser_Firefox,browser_Opera Touch,browser_Line,browser_MIUI Browser,browser_WeChat,browser_Samsung Browser,browser_Chrome WebView,browser_WebKit,browser_Android Browser,browser_Safari,browser_Opera,device_vendor_,device_vendor_OPPO,device_vendor_Apple,device_vendor_Realme,device_vendor_Vivo,device_vendor_Motorola,device_vendor_Sony,device_vendor_ASUS,device_vendor_OnePlus,device_vendor_HTC,device_vendor_Huawei,device_vendor_Google,device_vendor_Lenovo,device_vendor_Samsung,device_vendor_Nokia,device_vendor_LG,device_vendor_Xiaomi,device_vendor_Fairphone,is_ref_from_collections_page,is_ref_from_another_product_page,group
0,0,0,2022-03-23 13:19:20,pdpUniqueViewImage_0,66.249.73.101,0,3.999224e+11,7565723533535,m29252899832031,NaN,https%3A%2F%2Fwww.pe-nation.com%2Fcollections%...,US,NaN,NaN,NaN,37.751,-97.8220,Chrome,98.0,98.0.4758.102,Android,6.0.1,LG,Nexus 5,mobile,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,1,1,2022-03-23 13:35:20,pdpUniqueViewImage_0,101.166.122.46,1,3.999221e+11,7565748469983,m29253075632351,https%3A%2F%2Fwww.pe-nation.com%2Fcollections%...,https%3A%2F%2Fwww.pe-nation.com%2Fcollections%...,AU,NaN,NaN,NaN,-33.494,143.2104,Mobile Safari,15.0,15.3,iOS,15.3.1,Apple,iPhone,mobile,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,2,2,2022-03-23 13:35:22,pdpUniqueViewImage_1,101.166.122.46,1,3.999221e+11,7565748469983,m29253070782687,https%3A%2F%2Fwww.pe-nation.com%2Fcollections%...,https%3A%2F%2Fwww.pe-nation.com%2Fcollections%...,AU

In [ ]:
converts = data_df[data_df['conversion'] == 1]

In [ ]:
conversionDF = data_df[data_df['group'].isin(converts['group'])].reset_index(drop=True)

Fix conversion variable




In [ ]:
conversionDF['createdAt'] = pd.to_datetime(conversionDF['createdAt'])

In [ ]:
for group in conversionDF['group'].unique():
    groupDF = conversionDF[conversionDF['group'] == group]
    for i in range(0, len(groupDF)):    
        conversionDF.at[groupDF.index[i], 'conversion'] = 0
    conversionDF.at[groupDF.index[-1], 'conversion'] = 1


In [ ]:
conversionDF.head()

,Unnamed: 0,Unnamed: 0.1,createdAt,event,IP,Uid,CollectionID,ProductID,ImageID,Referer,URL,Country,Region,City,MetroArea,Latitude,Longitude,BrowserName,BrowserMajorVersion,BrowserVersion,OSName,OSVersion,DeviceVendor,DeviceModel,DeviceType,Background=Lifestyle,Background=Studio,BodyVisible=Both,BodyVisible=Bottom,BodyVisible=Top,Model=0,Model=1,Model=10,Model=101,Model=102,Model=103,Model=108,Model=109,Model=110,Model=112,Model=120,Model=121,Model=125,Model=135,Model=14,Model=15,Model=160,Model=19,Model=2,Model=200,Model=21,Model=23,Model=24,Model=25,Model=26,Model=27,Model=28,Model=29,Model=3,Model=31,Model=32,Model=33,Model=37,Model=38,Model=4,Model=41,Model=43,Model=44,Model=46,Model=48,Model=5,Model=51,Model=53,Model=58,Model=59,Model=6,Model=60,Model=66,Model=67,Model=68,Model=72,Model=74,Model=77,Model=78,Model=8,Model=80,Model=81,Model=83,Model=84,Model=85,Model=86,Model=9,Model=unknown,MoreThanOneModel=FALSE,MoreThanOneModel=TRUE,Pose=Back,Pose=Front,Pose=Side,Pose=Tilt (45),Position=Sitting,Position=Standing,ProductTag=Both,ProductTag=Bottom,ProductTag=N/A,ProductTag=Top,ProductTag_BodyVisible=Both_Both,ProductTag_BodyVisible=Both_Bottom,ProductTag_BodyVisible=Both_Top,ProductTag_BodyVisible=Bottom_Both,ProductTag_BodyVisible=Bottom_Bottom,ProductTag_BodyVisible=Bottom_Top,ProductTag_BodyVisible=N/A,ProductTag_BodyVisible=Top_Both,ProductTag_BodyVisible=Top_Bottom,ProductTag_BodyVisible=Top_Top,Recognizable=FALSE,Recognizable=TRUE,ShootingStyle=Flatlay,ShootingStyle=On-figure,UniqueCameraAngle=FALSE,UniqueCameraAngle=TRUE,UniquePose=FALSE,UniquePose=TRUE,conversion,browser_Chrome,browser_Mobile Safari,browser_Instagram,browser_Edge,browser_Facebook,browser_GSA,browser_Firefox,browser_Opera Touch,browser_Line,browser_MIUI Browser,browser_WeChat,browser_Samsung Browser,browser_Chrome WebView,browser_WebKit,browser_Android Browser,browser_Safari,browser_Opera,device_vendor_,device_vendor_OPPO,device_vendor_Apple,device_vendor_Realme,device_vendor_Vivo,device_vendor_Motorola,device_vendor_Sony,device_vendor_ASUS,device_vendor_OnePlus,device_vendor_HTC,device_vendor_Huawei,device_vendor_Google,device_vendor_Lenovo,device_vendor_Samsung,device_vendor_Nokia,device_vendor_LG,device_vendor_Xiaomi,device_vendor_Fairphone,is_ref_from_collections_page,is_ref_from_another_product_page,group
0,62,62,2022-03-23 15:07:18,pdpUniqueViewImage_0,124.148.220.108,16,3.999223e+11,7565768327391,m29255022936287,https%3A%2F%2Fwww.pe-nation.com%2Fcollections%...,https%3A%2F%2Fwww.pe-nation.com%2Fcollections%...,AU,WA,Morley,NaN,-31.8878,115.9085,Mobile Safari,15.0,15.3,iOS,15.3.1,Apple,iPhone,mobile,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,24
1,63,63,2022-03-23 15:07:20,pdpUniqueViewImage_1,124.148.220.108,16,3.999223e+11,7565768327391,m29255021986015,https%3A%2F%2Fwww.pe-nation.com%2Fcollections%...,https%3A%2F%2Fwww.pe-nation.com%2Fcollections%...,AU,WA,Morley,NaN,-31.8878,115.9085,Mobile Safari,15.0,15.3,iOS,15.3.1,Apple,iPhone,mobile,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,24
2,64,64,2022-03-23 15:07:21,pdpUniqueViewImage_2,124.148.220.108,16,3.999223e+11,7565768327391,m29255024443615,https%3A%2F%2Fwww.pe-nation.com%2Fcol

In [ ]:
print(data_df[data_df['conversion'] == 1].conversion.sum(), conversionDF[conversionDF['conversion'] == 1].conversion.sum())

5965 5965


## Create dataset for Image features from client

In [ ]:
ImageDF = conversionDF.iloc[:,7::]
ImageDF = ImageDF.drop(ImageDF.iloc[:, 2:18], axis=1)
ImageDF = ImageDF.drop(ImageDF.iloc[:, 101:-1], axis=1)
ImageDF.head()

,ProductID,ImageID,Background=Lifestyle,Background=Studio,BodyVisible=Both,BodyVisible=Bottom,BodyVisible=Top,Model=0,Model=1,Model=10,Model=101,Model=102,Model=103,Model=108,Model=109,Model=110,Model=112,Model=120,Model=121,Model=125,Model=135,Model=14,Model=15,Model=160,Model=19,Model=2,Model=200,Model=21,Model=23,Model=24,Model=25,Model=26,Model=27,Model=28,Model=29,Model=3,Model=31,Model=32,Model=33,Model=37,Model=38,Model=4,Model=41,Model=43,Model=44,Model=46,Model=48,Model=5,Model=51,Model=53,Model=58,Model=59,Model=6,Model=60,Model=66,Model=67,Model=68,Model=72,Model=74,Model=77,Model=78,Model=8,Model=80,Model=81,Model=83,Model=84,Model=85,Model=86,Model=9,Model=unknown,MoreThanOneModel=FALSE,MoreThanOneModel=TRUE,Pose=Back,Pose=Front,Pose=Side,Pose=Tilt (45),Position=Sitting,Position=Standing,ProductTag=Both,ProductTag=Bottom,ProductTag=N/A,ProductTag=Top,ProductTag_BodyVisible=Both_Both,ProductTag_BodyVisible=Both_Bottom,ProductTag_BodyVisible=Both_Top,ProductTag_BodyVisible=Bottom_Both,ProductTag_BodyVisible=Bottom_Bottom,ProductTag_BodyVisible=Bottom_Top,ProductTag_BodyVisible=N/A,ProductTag_BodyVisible=Top_Both,ProductTag_BodyVisible=Top_Bottom,ProductTag_BodyVisible=Top_Top,Recognizable=FALSE,Recognizable=TRUE,ShootingStyle=Flatlay,ShootingStyle=On-figure,UniqueCameraAngle=FALSE,UniqueCameraAngle=TRUE,UniquePose=FALSE,UniquePose=TRUE,conversion,group
0,7565768327391,m29255022936287,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0,24
1,7565768327391,m29255021986015,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0,24
2,7565768327391,m29255024443615,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0,24
3,7565768327391,m29255022608607,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0,24
4,7565768327391,m29255026770143,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0,24


Reduced features from client:

   } else if(shop == 'pe-nation-au.myshopify.com') {

    totalTagsK = ['Recognizable=FALSE', 'Recognizable=TRUE'];

    onlyTags = new Set(['ShootingStyle', 'Recognizable', 'Position', 
    'Pose', 'Background']);

In [ ]:
onlyTags = ['ShootingStyle', 'Recognizable', 'Position', 'Pose', 'Background']
column_list = ['ProductID', 'ImageID']
for tag in onlyTags:
    column_list.extend([col for col in data_df if col.startswith(tag)])
column_list.extend(['group', 'conversion', 'rate'])

In [ ]:
ImageDF = data_df[column_list]
ImageDF.head()

,ProductID,ImageID,ShootingStyle=Detail,ShootingStyle=Flatlay,ShootingStyle=On-figure,ShootingStyle=Detail_convert,ShootingStyle=Flatlay_convert,ShootingStyle=Detail_nonconvert,ShootingStyle=Flatlay_nonconvert,Recognizable=FALSE,Recognizable=TRUE,Recognizable=FALSE_convert,Recognizable=TRUE_convert,Recognizable=FALSE_nonconvert,Recognizable=TRUE_nonconvert,Position=N/A,Position=Sitting,Position=Standing,Position=N/A_convert,Position=Sitting_convert,Position=Standing_convert,Position=N/A_nonconvert,Position=Sitting_nonconvert,Position=Standing_nonconvert,Pose=Back,Pose=Front,Pose=Side,Pose=Tilt (45),Pose=Back_convert,Pose=Front_convert,Pose=Side_convert,Pose=Tilt (45)_convert,Pose=Back_nonconvert,Pose=Front_nonconvert,Pose=Side_nonconvert,Pose=Tilt (45)_nonconvert,Background=Lifestyle,Background=Studio,Background=Lifestyle_convert,Background=Studio_convert,Background=Lifestyle_nonconvert,Background=Studio_nonconvert,group,conversion,rate
0,7565723533535,m29252899832031,0.0,0.0,1.0,0.0,0.016503,0.0,0.013918,0.0,1.0,0.059197,0.239110,0.070416,0.257768,0.0,0.0,1.0,0.0,0.036314,0.245490,0.0,0.033630,0.280635,0.0,1.0,0.0,0.0,0.037377,0.191217,0.036944,0.016266,0.043957,0.209929,0.042682,0.017697,0.0,1.0,0.013352,0.284955,0.012663,0.315521,0,0,0.039370
1,7565748469983,m29253075632351,0.0,0.0,1.0,0.0,0.016503,0.0,0.013918,0.0,1.0,0.059197,0.239110,0.070416,0.257768,0.0,0.0,1.0,0.0,0.036314,0.245490,0.0,0.033630,0.280635,0.0,0.0,1.0,0.0,0.037377,0.191217,0.036944,0.016266,0.043957,0.209929,0.042682,0.017697,0.0,1.0,0.013352,0.284955,0.012663,0.315521,1,0,0.019704
2,7565748469983,m29253070782687,0.0,0.0,1.0,0.0,0.007089,0.0,0.007241,0.0,1.0,0.037456,0.138992,0.043240,0.146360,0.0,0.0,1.0,0.0,0.010949,0.158409,0.0,0.012210,0.170149,0.0,1.0,0.0,0.0,0.022332,0.117054,0.022489,0.007483,0.027703,0.122903,0.023859,0.007894,0.0,1.0,0.007798,0.168649,0.007350,0.182250,1,0,0.017582
3,7565748469983,m29253078122719,0.0,0.0,1.0,0.0,0.006341,0.0,0.006547,1.0,0.0,0.038322,0.114573,0.038926,0.115348,0.0,0.0,1.0,0.0,0.009571,0.136983,0.0,0.009881,0.137845,1.0,0.0,0.0,0.0,0.027609,0.088696,0.025404,0.004844,0.027978,0.090195,0.023918,0.005636,0.0,1.0,0.004529,0.148365,0.004575,0.149698,1,0,0.018018
4,7565763150047,m29252798742751,0.0,0.0,1.0,0.0,0.007089,0.0,0.007241,0.0,1.0,0.037456,0.138992,0.043240,0.146360,0.0,0.0,1.0,0.0,0.010949,0.158409,0.0,0.012210,0.170149,0.0,1.0,0.0,0.0,0.022332,0.117054,0.022489,0.007483,0.027703,0.122903,0.023859,0.007894,0.0,1.0,0.007798,0.168649,0.007350,0.182250,2,0,0.014620


In [ ]:
column_list=[]
column_list.extend([col for col in ImageDF if col.endswith('convert')])
ReducedImageDF = ImageDF.drop(column_list, axis=1)
ReducedImageDF.head()

,ProductID,ImageID,ShootingStyle=Detail,ShootingStyle=Flatlay,ShootingStyle=On-figure,Recognizable=FALSE,Recognizable=TRUE,Position=N/A,Position=Sitting,Position=Standing,Pose=Back,Pose=Front,Pose=Side,Pose=Tilt (45),Background=Lifestyle,Background=Studio,group,conversion,rate
0,7565723533535,m29252899832031,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0,0,0.039370
1,7565748469983,m29253075632351,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1,0,0.019704
2,7565748469983,m29253070782687,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1,0,0.017582
3,7565748469983,m29253078122719,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1,0,0.018018
4,7565763150047,m29252798742751,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,2,0,0.014620


Drop highly correlated columns from both Image Dataframe.

In [ ]:
ImageDF.iloc[:, 2:-2].corr()

,ShootingStyle=Detail,ShootingStyle=Flatlay,ShootingStyle=On-figure,ShootingStyle=Detail_convert,ShootingStyle=Flatlay_convert,ShootingStyle=Detail_nonconvert,ShootingStyle=Flatlay_nonconvert,Recognizable=FALSE,Recognizable=TRUE,Recognizable=FALSE_convert,Recognizable=TRUE_convert,Recognizable=FALSE_nonconvert,Recognizable=TRUE_nonconvert,Position=N/A,Position=Sitting,Position=Standing,Position=N/A_convert,Position=Sitting_convert,Position=Standing_convert,Position=N/A_nonconvert,Position=Sitting_nonconvert,Position=Standing_nonconvert,Pose=Back,Pose=Front,Pose=Side,Pose=Tilt (45),Pose=Back_convert,Pose=Front_convert,Pose=Side_convert,Pose=Tilt (45)_convert,Pose=Back_nonconvert,Pose=Front_nonconvert,Pose=Side_nonconvert,Pose=Tilt (45)_nonconvert,Background=Lifestyle,Background=Studio,Background=Lifestyle_convert,Background=Studio_convert,Background=Lifestyle_nonconvert,Background=Studio_nonconvert,group
ShootingStyle=Detail,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ShootingStyle=Flatlay,NaN,1.000000,-1.000000,NaN,0.049190,NaN,0.052889,-0.107921,0.107921,0.052346,0.051048,0.052196,0.050907,NaN,-0.053559,-0.535001,NaN,0.042456,0.052868,NaN,0.044529,0.051860,-0.084654,-0.237932,-0.073339,-0.041716,0.049856,0.050778,0.056068,0.040132,0.053530,0.050398,0.053516,0.044236,-0.035166,0.035166,0.048936,0.051503,0.049321,0.051264,0.020172
ShootingStyle=On-figure,NaN,-1.000000,1.000000,NaN,-0.049190,NaN,-0.052889,0.107921,-0.107921,-0.052346,-0.051048,-0.052196,-0.050907,NaN,0.053559,0.535001,NaN,-0.042456,-0.052868,NaN,-0.044529,-0.051860,0.084654,0.237932,0.073339,0.041716,-0.049856,-0.050778,-0.056068,-0.040132,-0.053530,-0.050398,-0.053516,-0.044236,0.035166,-0.035166,-0.048936,-0.051503,-0.049321,-0.051264,-0.020172
ShootingStyle=Detail_convert,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ShootingStyle=Flatlay_convert,NaN,0.049190,-0.049190,NaN,1.000000,NaN,0.993347,-0.073528,0.073528,0.958451,0.992115,0.983054,0.993204,NaN,0.071111,-0.088710,NaN,0.984847,0.979818,NaN,0.992738,0.988232,-0.082359,0.041575,-0.004343,0.015276,0.885460,0.987093,0.965270,0.962920,0.961509,0.989979,0.989658,0.987024,0.021005,-0.021005,0.981521,0.988771,0.986660,0.991963,0.095978
ShootingStyle=Detail_nonconvert,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ShootingStyle=Flatlay_nonconvert,NaN,0.052889,-0.052889,NaN,0.993347,NaN,1.000000,-0.076938,0.076938,0.958171,0.991824,0.984559,0.993368,NaN,0.068640,-0.088767,NaN,0.959120,0.985062,NaN,0.972694,0.991204,-0.085552,0.042906,-0.002896,0.012396,0.879902,0.987824,0.980889,0.934675,0.965829,0.989913,0.995847,0.967101,0.021720,-0.021720,0.982945,0.988384,0.987233,0.992380,0.098058
Recognizable=FALSE,NaN,-0.107921,0.107921,NaN,-0.073528,NaN,-0.076938,1.000000,-1.000000,-0.074815,-0.076699,-0.076909,-0.076808,NaN,-0.147604,0.187511,NaN,-0.066649,-0.078087,NaN,-0.069413,-0.077667,0.593716,-0.413403,-0.019531,0.021175,-0.067377,-0.077201,-0.077222,-0.067291,-0.076184,-0.076927,-0.076895,-0.070582,-0.103236,0.103236,-0.075964,-0.076563,-0.076344,-0.076889,0.035081
Recognizable=TRUE,NaN,0.107921,-0.107921,NaN,0.073528,NaN,0.076938,-1.000000,1.000000,0.074815,0.076699,0.076909,0.076808,NaN,0.147604,-0.187511,NaN,0.066649,0.078087,NaN,0.069413,0.077667,-0.593716,0.413403,0.019531,-0.021175,0.067377,0.077201,0.077222,0.067291,0.076184,0.076927,0.076895,0.070582,0.103236,-0.103236,0.075964,0.076563,0.076344,0.076889,-0.035081
Recognizable=FALSE_convert,NaN,0.052346,-0.052346,NaN,0.958451,NaN,0.958171,-0.074815,0.074815,1.000000,0.976790,0.990475,0.970928,NaN,0.066404,-0.086559,NaN,0.930009,0.986752,NaN,0.944731,0.977288,-0.080572,0.040757,-0.006321,0.014459,0.971950,0.

Drop highly correlated columns: ShootingStyle=Flatlay, Recognizable=FALSE, Position=Sitting, Background=Lifestyle, UniqueCameraAngle=TRUE

In [ ]:
ImageDF = ImageDF.drop(['ShootingStyle=Flatlay', 'Recognizable=FALSE', 'Position=Sitting', 'Background=Lifestyle', 'UniqueCameraAngle=TRUE'], axis=1)

In [ ]:
ReducedImageDF.iloc[:, 2:-2].corr()

,ShootingStyle=Detail,ShootingStyle=Flatlay,ShootingStyle=On-figure,Recognizable=FALSE,Recognizable=TRUE,Position=N/A,Position=Sitting,Position=Standing,Pose=Back,Pose=Front,Pose=Side,Pose=Tilt (45),Background=Lifestyle,Background=Studio,group
ShootingStyle=Detail,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ShootingStyle=Flatlay,NaN,1.000000,-1.000000,-0.107921,0.107921,NaN,-0.053559,-0.535001,-0.084654,-0.237932,-0.073339,-0.041716,-0.035166,0.035166,0.020172
ShootingStyle=On-figure,NaN,-1.000000,1.000000,0.107921,-0.107921,NaN,0.053559,0.535001,0.084654,0.237932,0.073339,0.041716,0.035166,-0.035166,-0.020172
Recognizable=FALSE,NaN,-0.107921,0.107921,1.000000,-1.000000,NaN,-0.147604,0.187511,0.593716,-0.413403,-0.019531,0.021175,-0.103236,0.103236,0.035081
Recognizable=TRUE,NaN,0.107921,-0.107921,-1.000000,1.000000,NaN,0.147604,-0.187511,-0.593716,0.413403,0.019531,-0.021175,0.103236,-0.103236,-0.035081
Position=N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position=Sitting,NaN,-0.053559,0.053559,-0.147604,0.147604,NaN,1.000000,-0.814984,-0.125387,0.191099,-0.086976,-0.031475,0.125002,-0.125002,0.010182
Position=Standing,NaN,-0.535001,0.535001,0.187511,-0.187511,NaN,-0.814984,1.000000,0.155212,-0.023607,0.116147,0.050839,-0.085352,0.085352,-0.020321
Pose=Back,NaN,-0.084654,0.084654,0.593716,-0.593716,NaN,-0.125387,0.155212,1.000000,-0.572873,-0.176580,-0.100442,-0.080219,0.080219,0.029464
Pose=Front,NaN,-0.237932,0.237932,-0.413403,0.413403,NaN,0.191099,-0.023607,-0.572873,1.000000,-0.496302,-0.282305,0.103506,-0.103506,-0.030778


Drop highly correlated columns: ShootingStyle=Flatlay, Recognizable=FALSE, Position=Sitting, Background=Lifestyle


In [ ]:
ReducedImageDF = ReducedImageDF.drop(['ShootingStyle=Flatlay', 'Recognizable=FALSE', 'Position=Sitting', 'Background=Lifestyle'], axis=1)

In [ ]:
ReducedImageDF.head()

,ProductID,ImageID,ShootingStyle=Detail,ShootingStyle=On-figure,Recognizable=TRUE,Position=N/A,Position=Standing,Pose=Back,Pose=Front,Pose=Side,Pose=Tilt (45),Background=Studio,group,conversion,rate
0,7565723533535,m29252899832031,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0,0,0.039370
1,7565748469983,m29253075632351,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1,0,0.019704
2,7565748469983,m29253070782687,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1,0,0.017582
3,7565748469983,m29253078122719,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1,0,0.018018
4,7565763150047,m29252798742751,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,2,0,0.014620


# Create Training/Val/Test Data

In [ ]:
def train_val_test(data: pd.DataFrame):
    gss = GroupShuffleSplit(test_size=.20, n_splits=1, random_state = 7).split(data, groups=data['ProductID'])
    X_train_inds, X_test_inds = next(gss)
    test_data= data.iloc[X_test_inds]
    train_data_1 = data.iloc[X_train_inds]

    gss = GroupShuffleSplit(test_size=.25, n_splits=1, random_state = 7).split(train_data_1, groups=train_data_1['ProductID'])
    X_train_inds, X_val_inds = next(gss)

    train_data= train_data_1.iloc[X_train_inds]
    X_train = train_data.loc[:, ~train_data.columns.isin(['group', 'ProductID','ImageID','conversion'])]
    y_train = train_data.loc[:, train_data.columns.isin(['conversion'])]
    val_data= train_data_1.iloc[X_val_inds]
    X_val = val_data.loc[:, ~val_data.columns.isin(['group','ProductID','ImageID','conversion'])]
    y_val = val_data.loc[:, val_data.columns.isin(['conversion'])]

    train_groups = train_data.groupby(['group']).size().to_frame('size')['size'].to_numpy()
    val_groups = val_data.groupby(['group']).size().to_frame('size')['size'].to_numpy()

    train_random = X_train.sample(frac=1).reset_index(drop=True)

  #We need to keep the id for later predictions
    X_test = test_data.loc[:, ~test_data.columns.isin(['conversion'])]
    y_test = test_data.loc[:, test_data.columns.isin(['conversion'])]

    print("Num train data points =", len(X_train))
    print("Num val data points =", len(X_val))
    print("Num test data points =", len(X_test))

    print("Num train conversions =", len(y_train[y_train["conversion"] == 1]))
    print("Num val conversions =", len(y_val[y_val["conversion"] == 1]))
    print("Num test conversions =", len(y_test[y_test["conversion"] == 1]))

    return (X_train, y_train, X_val, y_val, X_test, y_test, train_random, train_groups, val_groups)

In [ ]:
X_train_red, y_train_red, X_val_red, y_val_red, X_test_red, y_test_red, X_rand_red, train_groups_red, val_groups_red = train_val_test(ReducedImageDF)

Num train data points = 456019
Num val data points = 161995
Num test data points = 165582
Num train conversions = 14406
Num val conversions = 5587
Num test conversions = 5397


In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test, X_rand, train_groups, val_groups = train_val_test(ImageDF)

Num train data points = 13429
Num val data points = 3098
Num test data points = 3449
Num train conversions = 4123
Num val conversions = 943
Num test conversions = 899


# Build models with Image data




Results for reduced Image dataset:



In [ ]:
scale = len(y_train_red[y_train_red['conversion'] == 0])/len(y_train_red[y_train_red['conversion'] == 1])

estimator = xgb.XGBClassifier(
    objective= 'binary:logistic',
    nthread=4, eval_metric='auc',
    seed=42, tree_method='gpu_hist', scale_pos_weight = scale)

parameters = {
    'max_depth': [3, 5, 7, 9],
    'n_estimators': range(100, 300, 40), #make smaller based on previous thng
    'learning_rate': [0.005, 0.01, 0.05],
    'colsample_bytree': [0.4, 0.6, 0.8],
    'subsample': [0.8, 1.0]
}

In [ ]:
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'roc_auc',
    n_jobs = 4,
    cv = 4,
    verbose=True
)

In [ ]:
grid_search.fit(X_train_red, y_train_red)

Fitting 4 folds for each of 360 candidates, totalling 1440 fits


KeyboardInterrupt: ignored

In [ ]:
grid_search.best_estimator_

Best estimator:

XGBClassifier(colsample_bytree=0.6, eval_metric='auc', learning_rate=0.01,
              max_depth=5, n_estimators=200, nthread=4,
              scale_pos_weight=2.257094348775164, seed=42, subsample=1.0,
              tree_method='gpu_hist')

In [ ]:
bestXGBImageRed = xgb.XGBClassifier(colsample_bytree=0.8, eval_metric='auc', learning_rate=0.005,
              n_estimators=1000, nthread=4, scale_pos_weight=2.257094348775164,
              seed=42, subsample=1.0, tree_method='gpu_hist')

In [ ]:
bstImageRed = bestXGBImageRed.fit(X_train_red, y_train_red, eval_set=[(X_rand_red, y_train_red), (X_train_red, y_train_red), (X_val_red, y_val_red)],
    )

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


[0]	validation_0-auc:0.497337	validation_1-auc:0.676037	validation_2-auc:0.658303
[1]	validation_0-auc:0.498054	validation_1-auc:0.676089	validation_2-auc:0.65777
[2]	validation_0-auc:0.498034	validation_1-auc:0.675921	validation_2-auc:0.656683
[3]	validation_0-auc:0.498087	validation_1-auc:0.676312	validation_2-auc:0.657948
[4]	validation_0-auc:0.498104	validation_1-auc:0.676445	validation_2-auc:0.658228
[5]	validation_0-auc:0.498168	validation_1-auc:0.676596	validation_2-auc:0.65851
[6]	validation_0-auc:0.498373	validation_1-auc:0.676836	validation_2-auc:0.658875
[7]	validation_0-auc:0.498374	validation_1-auc:0.676863	validation_2-auc:0.658926
[8]	validation_0-auc:0.498557	validation_1-auc:0.677264	validation_2-auc:0.659261
[9]	validation_0-auc:0.498548	validation_1-auc:0.677232	validation_2-auc:0.659217
[10]	validation_0-auc:0.498524	validation_1-auc:0.677163	validation_2-auc:0.65918
[11]	validation_0-auc:0.498447	validation_1-auc:0.677229	validation_2-auc:0.659414
[12]	validation_0

In [ ]:
X_test_red = X_test_red.drop('ProductID', axis=1)

In [ ]:
test_results = pd.DataFrame(columns=['ImageID', 'group', 'prob'])
for group in X_test_red['group'].unique():
    ScoreDF = X_test_red[X_test_red['group'] == group]    
    group_test = pd.DataFrame(columns = test_results.columns)
    group_test['ImageID'] = ScoreDF['ImageID']
    group_test['group'] = ScoreDF['group']
    ScoreDF = ScoreDF.drop(['ImageID', 'group'], axis=1)
    group_test['prob'] = bstImageRed.predict_proba(ScoreDF)[:,1]
    group_test['conversion'] = bstImageRed.predict(ScoreDF)
    test_results = test_results.append(group_test)

In [ ]:
test_results.head()

,ImageID,group,prob,conversion
27,m29254795329759,7,0.152088,0.0
28,m29254794248415,7,0.170273,0.0
41,m29253185732831,17,0.050694,0.0
51,m29253154996447,21,0.008199,0.0
52,m29287207829727,22,0.079507,0.0


In [ ]:
print("Accuracy:  " + str(accuracy_score(y_test_red, test_results['conversion'])) + \
      "\nPrecision: " + str(precision_score(y_test_red, test_results['conversion'])) + \
      "\nRecall:    " + str(recall_score(y_test_red, test_results['conversion'])) + \
      "\nF1 Score:  " + str(f1_score(y_test_red, test_results['conversion'])))

Accuracy:  0.9674058774504475
Precision: 0.0
Recall:    0.0
F1 Score:  0.0


Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.


0.0    165582
Name: conversion, dtype: int64